## Importing libraries and dataset

In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Embedding
from sklearn.model_selection import train_test_split
print("Modules imported")

Modules imported


In [2]:
df = pd.read_csv('train.csv', parse_dates=['date'], index_col = ['date'])
df.head()

,store,item,sales
date,,,
2013-01-01,1,1,13
2013-01-02,1,1,11
2013-01-03,1,1,14
2013-01-04,1,1,13
2013-01-05,1,1,10


In [3]:
df = df.sort_values('date',ascending=True)
test = df[df.index.year == 2017]
test.reset_index(level=0, inplace= True)
train = df[df.index.year != 2017]
train.reset_index(level = 0, inplace = True)
train.head()

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-01,7,12,26
2,2013-01-01,7,46,27
3,2013-01-01,8,12,54
4,2013-01-01,9,12,35


# Dataset pre-processing

In [4]:
train_data = pd.DataFrame({'year': train['date'].dt.year-2013, 'month': train['date'].dt.month,
                           'day': train['date'].dt.day, 'weekday': train['date'].dt.weekday,
                           'store': train['store'], 'item': train['item'], 'sales': train['sales']},
                          columns =['year', 'month', 'day', 'weekday', 'store', 'item', 'sales'])

test_data = pd.DataFrame({'year': test['date'].dt.year-2013, 'month': test['date'].dt.month,
                           'day': test['date'].dt.day, 'weekday': test['date'].dt.weekday,
                           'store': test['store'], 'item': test['item'], 'sales': test['sales']},
                          columns =['year', 'month', 'day', 'weekday', 'store', 'item', 'sales'])


In [5]:
print(train_data.head())
print(train_data.shape)

   year  month  day  weekday  store  item  sales
0     0      1    1        1      1     1     13
1     0      1    1        1      7    12     26
2     0      1    1        1      7    46     27
3     0      1    1        1      8    12     54
4     0      1    1        1      9    12     35
(730500, 7)


In [6]:
X = np.array(train_data.drop('sales', axis = 1))
y = np.array(train_data['sales'])
X_test = np.array(test_data.drop('sales', axis = 1))
y_test = np.array(test_data['sales'])

In [7]:

def split_data(X_train, y_train, val_ratio = 0.2, val_year = 3, half_yearly = 1, randomly = True):
    
    # Splitting randomly
    if randomly:
        X_tr, y_tr, X_val, y_val = train_test_split(X_train, y_train, test_size = (val_ratio),
                                                          random_state = 6, shuffle = True)
    else:
        if half_yearly == 1:        #if validation data is first 6 months of val_year
            
            X_tr = X_train[(X_train[:,0]!=val_year) | (X_train[:,1]>6)]   #if not val_year or in last 6 months of year
            y_tr = y_train[(X_train[:,0]!=val_year) | (X_train[:,1]>6)]
            
            X_val = X_train[(X_train[:,0]==val_year) & (X_train[:,1]<=6)] #if val_year and first 6 months of year
            y_val = y_train[(X_train[:,0]==val_year) & (X_train[:,1]<=6)]
            
        else:                       #if validation data is last 6 months of val_year
            
            X_tr = X_train[(X_train[:,0]!=val_year) | (X_train[:,1]<=6)]  #if not val_year or in first 6 months of year
            y_tr = y_train[(X_train[:,0]!=val_year) | (X_train[:,1]<=6)]
            
            X_val = X_train[(X_train[:,0]==val_year) & (X_train[:,1]>6)]  #if val_year and last 6 months of year
            y_val = y_train[(X_train[:,0]==val_year) & (X_train[:,1]>6)]
            
        return X_tr, y_tr, X_val, y_val

In [8]:
X_train, y_train, X_val, y_val = split_data(X, y, False, 0.3, 3, 0)
print("Training:", X_train.shape, y_train.shape)
print("Validation:", X_val.shape, y_val.shape)

Training: (730500, 6) (730500,)
Validation: (0, 6) (0,)


# Creating FeedForward Neural Network model

In [9]:
# For embeddings, the thumb rule is, num_embeddings = no. of unique valus in category + 1 
# & embedding_dim = min(50,feat_dim(num_embeddings)/2)
dims = [np.unique(X_train[:,i]).size+1 for i in range(X_train.shape[1])]
print(dims)
embedding_dim = [(x, min(50, (x+1)//2)) for x in dims]
print(embedding_dim)

[5, 13, 32, 8, 11, 51]
[(5, 3), (13, 7), (32, 16), (8, 4), (11, 6), (51, 26)]


In [10]:
# Creating class for Feed Forward Neural Network
class NNwithEmbeddings(nn.Module):
    def __init__(self, embedding_dim, n_cont, out_size, layers, dp = 0.3):    #n_cont=no. of cont. feat. in dataframe
        super().__init__()
        self.embeds = nn.ModuleList([nn.Embedding(inp,out) for inp,out in embedding_dim])
        #print(self.embeds)
        self.emb_drop = nn.Dropout(dp)
        
        layer_list = []
        n_emb = sum((out for inp,out in embedding_dim))
        n_in = n_emb + n_cont
        #print(n_in)
        
        for i in layers:
            layer_list.append(nn.Linear(n_in, i))
            layer_list.append(nn.ReLU(inplace = True))
            #layer_list.append(nn.BatchNorm1d(i))
            layer_list.append(nn.Dropout(dp))
            n_in = i
        layer_list.append(nn.Linear(layers[-1], out_size))
        
        self.layers = nn.Sequential(*layer_list)
        
    def forward(self, X_cat, X_cont):
        embeddings = []
        for i, e in enumerate(self.embeds):
            #print(i,e,X_cat[:5,i])
            embeddings.append(e(X_cat[:,i]))
        X = torch.cat(embeddings, axis =1)
        X = self.emb_drop(X)

        X = torch.cat([X, torch.unsqueeze(X_cont,1)], 1)
        out = self.layers(X)

        return out
    
    def preprocess(self, X):
        return split_features(X)


In [11]:
# Initiating neural network model with 3 hidden layers having 512,128 & 32 nodes each
NNmodel = NNwithEmbeddings(embedding_dim[1:], 1, 1,[512,128,32], 0)
NNmodel

NNwithEmbeddings(
  (embeds): ModuleList(
    (0): Embedding(13, 7)
    (1): Embedding(32, 16)
    (2): Embedding(8, 4)
    (3): Embedding(11, 6)
    (4): Embedding(51, 26)
  )
  (emb_drop): Dropout(p=0, inplace=False)
  (layers): Sequential(
    (0): Linear(in_features=60, out_features=512, bias=True)
    (1): ReLU(inplace=True)
    (2): Dropout(p=0, inplace=False)
    (3): Linear(in_features=512, out_features=128, bias=True)
    (4): ReLU(inplace=True)
    (5): Dropout(p=0, inplace=False)
    (6): Linear(in_features=128, out_features=32, bias=True)
    (7): ReLU(inplace=True)
    (8): Dropout(p=0, inplace=False)
    (9): Linear(in_features=32, out_features=1, bias=True)
  )
)

In [12]:
# custom loss function (for optional use)
def smape(x,y):
    return 100*torch.mean(2*torch.abs(x-y)/(torch.abs(x)+torch.abs(y)))

optim = torch.optim.Adam(NNmodel.parameters(), lr = 0.01)       # We use Adam optimizer and Mean squared Error
lossfn = F.mse_loss                                             # loss for training

# Training

In [13]:
def fit(epochs, sets, model, X_train, y_train, lossfn, optimizer):
    for i in range(sets//2):
        for j in range(2):
            X_tr, y_tr, X_val, y_val = split_data(X_train, y_train, val_year=i, half_yearly=j, randomly=False)
            losses = []
            for k in range(epochs):
                k+=1
                y_pred = NNmodel(torch.from_numpy(X_tr[:,1:]), torch.from_numpy(X_tr[:,0]))
                y_tr = torch.tensor(y_tr,dtype=torch.float).reshape(-1,1)
                loss = lossfn(y_pred,torch.tensor(y_tr))
                losses.append(loss)
                #if k%2 == 1:
                print("Epoch number {} of validation year 20{} and half {} has MSE loss {}".format(k,13+i,j,loss.item()))
                    
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()

                if k%2 == 0:
                    with torch.no_grad():
                        yhat_val = NNmodel(torch.from_numpy(X_val[:,1:]), torch.from_numpy(X_val[:,0]))
                        y_val = torch.tensor(y_val,dtype=torch.float).reshape(-1,1)
                        val_loss = torch.sqrt(lossfn(torch.tensor(y_val), yhat_val))
                    print("Validation loss at epoch {} of year 20{} half {} is {:.4f}".format(k,13+i,j,val_loss.item()))


In [14]:
import time
begin = time.time()
fit(32, 8, NNmodel, X, y, lossfn, optim)
end = time.time()


/tmp/ipykernel_5301/3281674988.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  loss = lossfn(y_pred,torch.tensor(y_tr))


Epoch number 1 of validation year 2013 and half 0 has MSE loss 3435.637451171875


/tmp/ipykernel_5301/3281674988.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_tr = torch.tensor(y_tr,dtype=torch.float).reshape(-1,1)


Epoch number 2 of validation year 2013 and half 0 has MSE loss 3194.018310546875


/tmp/ipykernel_5301/3281674988.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  val_loss = torch.sqrt(lossfn(torch.tensor(y_val), yhat_val))


...

/tmp/ipykernel_5301/3281674988.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_val = torch.tensor(y_val,dtype=torch.float).reshape(-1,1)


Validation loss at epoch 4 of year 2013 half 0 is 23.9613
Epoch number 5 of validation year 2013 and half 0 has MSE loss 778.6845703125
Epoch number 6 of validation year 2013 and half 0 has MSE loss 1949.109619140625
Validation loss at epoch 6 of year 2013 half 0 is 31.4415
Epoch number 7 of validation year 2013 and half 0 has MSE loss 1119.71826171875
Epoch number 8 of validation year 2013 and half 0 has MSE loss 612.288330078125
Validation loss at epoch 8 of year 2013 half 0 is 21.7779
Epoch number 9 of validation year 2013 and half 0 has MSE loss 695.399658203125
Epoch number 10 of validation year 2013 and half 0 has MSE loss 886.1011962890625
Validation loss at epoch 10 of year 2013 half 0 is 25.5493
Epoch number 11 of validation year 2013 and half 0 has MSE loss 969.1436157226562
Epoch number 12 of validation year 2013 and half 0 has MSE loss 913.7047119140625
Validation loss at epoch 12 of year 2013 half 0 is 22.1434
Epoch number 13 of validation year 2013 and half 0 has MSE loss

Validation loss at epoch 14 of year 2014 half 0 is 8.6136
Epoch number 15 of validation year 2014 and half 0 has MSE loss 73.89508819580078
Epoch number 16 of validation year 2014 and half 0 has MSE loss 73.08456420898438
Validation loss at epoch 16 of year 2014 half 0 is 8.6316
Epoch number 17 of validation year 2014 and half 0 has MSE loss 72.37355041503906
Epoch number 18 of validation year 2014 and half 0 has MSE loss 71.73001861572266
Validation loss at epoch 18 of year 2014 half 0 is 8.6048
Epoch number 19 of validation year 2014 and half 0 has MSE loss 71.06254577636719
Epoch number 20 of validation year 2014 and half 0 has MSE loss 70.37023162841797
Validation loss at epoch 20 of year 2014 half 0 is 8.4783
Epoch number 21 of validation year 2014 and half 0 has MSE loss 69.72797393798828
Epoch number 22 of validation year 2014 and half 0 has MSE loss 69.15701293945312
Validation loss at epoch 22 of year 2014 half 0 is 8.3834
Epoch number 23 of validation year 2014 and half 0 has

Validation loss at epoch 24 of year 2015 half 0 is 7.6949
Epoch number 25 of validation year 2015 and half 0 has MSE loss 56.42906951904297
Epoch number 26 of validation year 2015 and half 0 has MSE loss 56.35916519165039
Validation loss at epoch 26 of year 2015 half 0 is 7.6907
Epoch number 27 of validation year 2015 and half 0 has MSE loss 56.29690933227539
Epoch number 28 of validation year 2015 and half 0 has MSE loss 56.22747802734375
Validation loss at epoch 28 of year 2015 half 0 is 7.6796
Epoch number 29 of validation year 2015 and half 0 has MSE loss 56.1566162109375
Epoch number 30 of validation year 2015 and half 0 has MSE loss 56.09492874145508
Validation loss at epoch 30 of year 2015 half 0 is 7.6702
Epoch number 31 of validation year 2015 and half 0 has MSE loss 56.03496170043945
Epoch number 32 of validation year 2015 and half 0 has MSE loss 55.96944808959961
Validation loss at epoch 32 of year 2015 half 0 is 7.6657
Epoch number 1 of validation year 2015 and half 1 has M

Validation loss at epoch 2 of year 2016 half 1 is 7.5626
Epoch number 3 of validation year 2016 and half 1 has MSE loss 53.54899597167969
Epoch number 4 of validation year 2016 and half 1 has MSE loss 53.5174674987793
Validation loss at epoch 4 of year 2016 half 1 is 7.5600
Epoch number 5 of validation year 2016 and half 1 has MSE loss 53.48748779296875
Epoch number 6 of validation year 2016 and half 1 has MSE loss 53.45970916748047
Validation loss at epoch 6 of year 2016 half 1 is 7.5605
Epoch number 7 of validation year 2016 and half 1 has MSE loss 53.433982849121094
Epoch number 8 of validation year 2016 and half 1 has MSE loss 53.4094123840332
Validation loss at epoch 8 of year 2016 half 1 is 7.5617
Epoch number 9 of validation year 2016 and half 1 has MSE loss 53.385162353515625
Epoch number 10 of validation year 2016 and half 1 has MSE loss 53.36107635498047
Validation loss at epoch 10 of year 2016 half 1 is 7.5633
Epoch number 11 of validation year 2016 and half 1 has MSE loss 5

In [21]:
print("Time taken for training in a Ryzen 5 Hexa core 4600H CPU is {:.2f} minutes".format((end-begin)/60))

Time taken for training in a Ryzen 5 Hexa core 4600H CPU is 30.64 minutes



# Testing

In [16]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
#import torch_metrics as tm

In [17]:
def show_metrics(test, y_pred_final):
    metrics = {'R2_score': r2_score(test, y_pred_final), 'MAE': mean_absolute_error(test, y_pred_final),
               'RMSE': mean_squared_error(test, y_pred_final, squared=False),
               'MAPE': mean_absolute_percentage_error(test, y_pred_final)}
    adj_R2 = 1-(1-metrics['R2_score'])*(len(test)-1)/(len(test)-6-1)      #num of indep var = 6
    metrics['adj_R2'] = adj_R2
    print("R2 score on test set is", metrics['R2_score'])
    print("Mean Absolute Error on test set is", metrics['MAE'])
    print("Root Mean Square error on test set is", metrics['RMSE'])
    print("Mean Absolute Percentage Error on test set is", metrics['MAPE'])
    print("Adjusted R2 score on test set is", adj_R2)
    
    return metrics

In [18]:
predictions = NNmodel(torch.from_numpy(X_test[:,1:]), torch.from_numpy(X_test[:,0]))
test_results = show_metrics(y_test, predictions.detach().numpy())

R2 score on test set is 0.923525640363284
Mean Absolute Error on test set is 6.846392510761627
Root Mean Square error on test set is 8.725558357325898
Mean Absolute Percentage Error on test set is 0.15158092560938577
Adjusted R2 score on test set is 0.9235231260413219


# Saving the model

In [19]:
torch.save(NNmodel.state_dict(), 'FFNN.pth')